In [0]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName('Prabhakar').getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
df = spark.read.option("multiline",'true').json("/FileStore/tables/tripdetail_json-3.json")

In [0]:
df.show()

+------------+------------+----------+--------+--------------------+-----+-----------+---------------------+-------+---------------+------------+------------------+----------+------------+------------+---------------------+--------------------+-------------+
|DOLocationID|PULocationID|RatecodeID|VendorID|congestion_surcharge|extra|fare_amount|improvement_surcharge|mta_tax|passenger_count|payment_type|store_and_fwd_flag|tip_amount|tolls_amount|total_amount|tpep_dropoff_datetime|tpep_pickup_datetime|trip_distance|
+------------+------------+----------+--------+--------------------+-----+-----------+---------------------+-------+---------------+------------+------------------+----------+------------+------------+---------------------+--------------------+-------------+
|          43|         142|         1|       1|                 2.5|  3.0|        8.0|                  0.3|    0.5|              1|           2|                 N|       0.0|         0.0|        11.8|  2021-01-01 00:36:12|

In [0]:
###2. Convert the tpep_pickup_datetime, tpep_dropoff_datetime both columns into IST from PST and add as a seperate column with _IST suffix.

In [0]:
from pyspark.sql import functions as F



df2 = df.withColumn('tpep_dropoff_datetime_IST', df.tpep_dropoff_datetime.cast('timestamp'))
df3 = df2.withColumn('tpep_dropoff_datetime_IST', F.from_utc_timestamp(df2.tpep_dropoff_datetime_IST, "EST"))
df3.show()
df4= df3.withColumn('tpep_pickup_datetime_IST', df3.tpep_pickup_datetime.cast('timestamp'))
df5= df4.withColumn('tpep_pickup_datetime_IST', F.from_utc_timestamp(df4.tpep_pickup_datetime_IST, "EST"))
df5.show()

+------------+------------+----------+--------+--------------------+-----+-----------+---------------------+-------+---------------+------------+------------------+----------+------------+------------+---------------------+--------------------+-------------+-------------------------+
|DOLocationID|PULocationID|RatecodeID|VendorID|congestion_surcharge|extra|fare_amount|improvement_surcharge|mta_tax|passenger_count|payment_type|store_and_fwd_flag|tip_amount|tolls_amount|total_amount|tpep_dropoff_datetime|tpep_pickup_datetime|trip_distance|tpep_dropoff_datetime_IST|
+------------+------------+----------+--------+--------------------+-----+-----------+---------------------+-------+---------------+------------+------------------+----------+------------+------------+---------------------+--------------------+-------------+-------------------------+
|          43|         142|         1|       1|                 2.5|  3.0|        8.0|                  0.3|    0.5|              1|           2|

In [0]:
##3. Add addition column TravelTime by finding the difference between tpep_pickup_datetime and tpep_dropoff_datetime dates

In [0]:
df5.withColumn("TravelTime", df5.tpep_pickup_datetime*0.3) \
  .show()

+------------+------------+----------+--------+--------------------+-----+-----------+---------------------+-------+---------------+------------+------------------+----------+------------+------------+---------------------+--------------------+-------------+-------------------------+------------------------+-------------+----------+
|DOLocationID|PULocationID|RatecodeID|VendorID|congestion_surcharge|extra|fare_amount|improvement_surcharge|mta_tax|passenger_count|payment_type|store_and_fwd_flag|tip_amount|tolls_amount|total_amount|tpep_dropoff_datetime|tpep_pickup_datetime|trip_distance|tpep_dropoff_datetime_IST|tpep_pickup_datetime_IST|DiffInSeconds|TravelTime|
+------------+------------+----------+--------+--------------------+-----+-----------+---------------------+-------+---------------+------------+------------------+----------+------------+------------+---------------------+--------------------+-------------+-------------------------+------------------------+-------------+-------

In [0]:
df3 = df.withColumn("TravelTime",(to_timestamp(col("tpep_dropoff_datetime")).cast("long"))- to_timestamp(col("tpep_pickup_datetime")).cast("long"))
df3.show()

+------------+------------+----------+--------+--------------------+-----+-----------+---------------------+-------+---------------+------------+------------------+----------+------------+------------+---------------------+--------------------+-------------+----------+
|DOLocationID|PULocationID|RatecodeID|VendorID|congestion_surcharge|extra|fare_amount|improvement_surcharge|mta_tax|passenger_count|payment_type|store_and_fwd_flag|tip_amount|tolls_amount|total_amount|tpep_dropoff_datetime|tpep_pickup_datetime|trip_distance|TravelTime|
+------------+------------+----------+--------+--------------------+-----+-----------+---------------------+-------+---------------+------------+------------------+----------+------------+------------+---------------------+--------------------+-------------+----------+
|          43|         142|         1|       1|                 2.5|  3.0|        8.0|                  0.3|    0.5|              1|           2|                 N|       0.0|         0.0|  

In [0]:
df5.toPandas().to_csv('my_csv.csv')

In [0]:
df2 = df.withColumn("tpep_pickup_datetime_IST",df.tpep_pickup_datetime + expr('INTERVAL 12 HOURS 30 minutes'))
df2 = df2.withColumn("tpep_pickup_datetime_IST",df.tpep_pickup_datetime + expr('INTERVAL 12 HOURS 30 minutes'))
df2.select("tpep_pickup_datetime_IST","tpep_dropoff_datetime_IST").show()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-3027402770206635> in <module>
----> 1 df2 = df.withColumn("tpep_pickup_datetime_IST",df.tpep_pickup_datetime + expr('INTERVAL 12 HOURS 30 minutes'))
      2 df2 = df2.withColumn("tpep_pickup_datetime_IST",df.tpep_pickup_datetime + expr('INTERVAL 12 HOURS 30 minutes'))
      3 df2.select("tpep_pickup_datetime_IST","tpep_dropoff_datetime_IST").show()

NameError: name 'expr' is not defined

In [0]:
##4. Write the dataframe into csv files (as 3 partition)

In [0]:
df5.write.option("header",True).partitionBy("VendorID").mode("overwrite").csv("/FileStore/tables/vendor1.csv")

In [0]:
##5. Build a temp view on top of the data frame

In [0]:
df.createOrReplaceTempView("Tripdetails")
df2 = spark.sql("SELECT * from Tripdetails")
df2.printSchema()

root
 |-- DOLocationID: long (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- VendorID: long (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- trip_distance: string (nullable = true)

+------------+------------+----------+--------+--------------------+-----+-----------+---------------------+-------+---------------+------------+------------------+----------+------------+---

In [0]:
df2.show()

+------------+------------+----------+--------+--------------------+-----+-----------+---------------------+-------+---------------+------------+------------------+----------+------------+------------+---------------------+--------------------+-------------+
|DOLocationID|PULocationID|RatecodeID|VendorID|congestion_surcharge|extra|fare_amount|improvement_surcharge|mta_tax|passenger_count|payment_type|store_and_fwd_flag|tip_amount|tolls_amount|total_amount|tpep_dropoff_datetime|tpep_pickup_datetime|trip_distance|
+------------+------------+----------+--------+--------------------+-----+-----------+---------------------+-------+---------------+------------+------------------+----------+------------+------------+---------------------+--------------------+-------------+
|          43|         142|         1|       1|                 2.5|  3.0|        8.0|                  0.3|    0.5|              1|           2|                 N|       0.0|         0.0|        11.8|  2021-01-01 00:36:12|

In [0]:
##6. Write spark sql query to aggregate fare_amount based on Vendor Id.

In [0]:
df2 = spark.sql("SELECT VendorID,sum(fare_amount) from tripdetails group by VendorID")

In [0]:
df2.show()

+--------+------------------+
|VendorID|  sum(fare_amount)|
+--------+------------------+
|       1|2230.1800000000003|
|       2|            4128.0|
+--------+------------------+



In [0]:
##7. write the output into single CSV file.
df2.toPandas().to_csv("Mycsv425.csv")

In [0]:
df2

Out[84]: DataFrame[VendorID: bigint, sum(fare_amount): double]